In [1]:
import argparse
import math
import socket
import importlib
import os
import pickle

from datetime import datetime
#import h5pyprovider
import numpy as np
import tensorflow as tf
from keras import backend as K
import matplotlib.pylab as plt
# os.environ['CUDA_DEVICE_ORDER']="PCI_BUS_ID"
# os.environ['CUDA_VISIBLE_DEVICES']="1"

import sys
BASE_DIR = os.path.abspath('')
print(BASE_DIR)
sys.path.append(os.path.join(BASE_DIR, 'models'))
sys.path.append(BASE_DIR) # model
sys.path.append(os.path.join(BASE_DIR, 'tf_utils'))
import provider
import tf_util

/home/michael/python-virtual-environments/pointscnn/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:469: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/michael/python-virtual-environments/pointscnn/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:470: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/michael/python-virtual-environments/pointscnn/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:471: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 

/home/michael/lidar/D-FCN


Using TensorFlow backend.


In [2]:
# Argparses
EPOCH_CNT = 0

BATCH_SIZE = 6
NUM_POINT = 2048
MAX_EPOCH = 200
BASE_LEARNING_RATE = .01
# GPU_INDEX = 7
GPU_INDEX = 1
MOMENTUM = 0.9
OPTIMIZER = 'adam'
DECAY_STEP = 20000
DECAY_RATE = 0.5

In [3]:
MODEL = importlib.import_module('DFCN_pointnet2_group2') # import network module
MODEL_FILE = os.path.join(BASE_DIR, 'DFCN_pointnet2_group2'+'.py')
LOG_DIR = 'log_wen_v16_sample8192_group2_lw14_F1_noheight'
if not os.path.exists(LOG_DIR): os.mkdir(LOG_DIR)
os.system('cp models/DFCN_pointnet2.py %s' % (LOG_DIR)) # bkp of model def
os.system('cp tf_utils/DFCN_util_xy2.py %s' % (LOG_DIR)) # bkp of model def
# os.system('cp train_pointsift_lx_npsplit_V16.ipynb %s' % (LOG_DIR)) # bkp of train procedure
LOG_FOUT = open(os.path.join(LOG_DIR, 'log_train.txt'), 'w')
# LOG_FOUT.write(str(FLAGS)+'\n')

BN_INIT_DECAY = 0.5
BN_DECAY_DECAY_RATE = 0.5
BN_DECAY_DECAY_STEP = float(DECAY_STEP)
BN_DECAY_CLIP = 0.99

HOSTNAME = socket.gethostname()

NUM_CLASSES = 9

BASE_DIR /home/michael/lidar/D-FCN/tf_utils


In [4]:
def log_string(out_str):
    LOG_FOUT.write(out_str+'\n')
    LOG_FOUT.flush()
    print(out_str)

def get_learning_rate(batch):
    learning_rate = tf.train.exponential_decay(
                        BASE_LEARNING_RATE,  # Base learning rate.
                        batch * BATCH_SIZE,  # Current index into the dataset.
                        DECAY_STEP,          # Decay step.
                        DECAY_RATE,          # Decay rate.
                        staircase=True)
    learning_rate = tf.maximum(learning_rate, 0.00001) # CLIP THE LEARNING RATE!!
    return learning_rate        

def get_bn_decay(batch):
    bn_momentum = tf.train.exponential_decay(
                      BN_INIT_DECAY,
                      batch*BATCH_SIZE,
                      BN_DECAY_DECAY_STEP,
                      BN_DECAY_DECAY_RATE,
                      staircase=True)
    bn_decay = tf.minimum(BN_DECAY_CLIP, 1 - bn_momentum)
    return bn_decay

def Acc_from_confusions(confusions):
    
    TP = np.diagonal(confusions, axis1=-2, axis2=-1)
    TP_plus_FN = np.sum(confusions, axis=-1)
    TP_plus_FP = np.sum(confusions, axis=-2)
    
    mAcc = np.sum(TP)/np.sum(confusions)
    
    precision = TP / (TP_plus_FP + 1e-6)
    recall = TP / (TP_plus_FN+ 1e-6)
    fscore = 2*(precision * recall)/(precision + recall + 1e-6)
    
    ave_F1 = np.mean(fscore)
    
    s = 'Overall accuracy：{:5.2f}  Average F1 score：{:5.2f} \n'.format(100 * mAcc, 100 * ave_F1)
    s += log_acc(precision)
    s += log_acc(recall)
    s += log_acc(fscore)
    
    log_string(s)
    
    return mAcc, ave_F1
    

def log_acc(acc_list):
    s = ""
    for acc in acc_list:
        s += '{:5.2f} '.format(100 * acc)
    s += '\n'
    return s

def pc_normalize(pc):
    l = pc.shape[0]
    centroid = np.mean(pc, axis=0)
    pc = pc - centroid
    m = np.max(np.sqrt(np.sum(pc**2, axis=1)))
    pc = pc / m
    return pc
    
def drawPlot(x,y,name):
    plt.rcParams['savefig.dpi'] = 300 
    plt.plot(np.arange(0,len(x)),x,'k-',alpha=1,label='Train max: '+str(round(max(x),3))+', min: '+str(round(min(x),3)))
    plt.plot(np.arange(0,len(y)),y,'r-',alpha=1,label='Test max: '+str(round(max(y),3))+', min: '+str(round(min(y),3)))
    plt.legend()
    plt.xlabel('epoch',fontsize=9)
    plt.ylabel(name+' value',fontsize=9)
    plt.savefig(LOG_DIR+"/"+name+".png",bbox_inches='tight')
    plt.show()
    
def drawF1Plot(x,name):
    plt.rcParams['savefig.dpi'] = 300 
    plt.plot(np.arange(0,len(x)),x,'k-',alpha=1,label='Train max: '+str(round(max(x),3))+', min: '+str(round(min(x),3)))
    plt.legend()
    plt.xlabel('epoch',fontsize=9)
    plt.ylabel(name+' value',fontsize=9)
    plt.savefig(LOG_DIR+"/"+name+".png",bbox_inches='tight')
    plt.show()
    
    
def placeholder_inputs(batch_size, num_point):
    pointclouds_pl = tf.placeholder(tf.float32, shape=(None, None, 3))
    labels_pl = tf.placeholder(tf.int32, shape=(None, None))
    smpws_pl = tf.placeholder(tf.float32, shape=(None, None))
    return pointclouds_pl, labels_pl, smpws_pl

def pc_normalize_min_max(data):
    mindata = np.min(data[:,:3], axis=0)
    maxdata = np.max(data[:,:3], axis=0)
    return 2*(data[:,:3] - mindata)/(maxdata - mindata)

def pc_normalize_min(data):
    mindata = np.min(data[:,:3], axis=0)
    
    return (data[:,:3] - mindata)

def get_batch(dataset, index, npoints = NUM_POINT):
  
    if(dataset =='train'):
        cub_l = 30.0
        cub_w = 30.0
        cub_h = 100.0
        point_set =  trainSet[:,:3] - np.min(trainSet[:,:3], axis=0)
        semantic_seg = trainSet[:,4].astype(np.int32)
        coordmax = np.max(point_set,axis=0)
        coordmin = np.min(point_set,axis=0)
        smpmin = np.maximum(coordmax-[cub_l,cub_w,cub_h], coordmin)
        smpmin[2] = coordmin[2]
        smpsz = np.minimum(coordmax-smpmin,[cub_l,cub_w,cub_h])
        smpsz[2] = coordmax[2]-coordmin[2]
        isvalid = False
        for i in range(10):
            curcenter = point_set[np.random.choice(len(semantic_seg),1)[0],:]
            curmin = curcenter-[cub_l/2,cub_w/2,cub_h/2]
            curmax = curcenter+[cub_l/2,cub_w/2,cub_h/2]
            curmin[2] = coordmin[2]
            curmax[2] = coordmax[2]
            curchoice = np.sum((point_set>=(curmin-0.0))*(point_set<(curmax+0.0)),axis=1)==3
            cur_point_set = point_set[curchoice,:]
            cur_semantic_seg = semantic_seg[curchoice]
            cur_feat_set = trainFeats[curchoice,:]
    #         if len(cur_semantic_seg)<npoints:
            if len(cur_semantic_seg)==0:
                continue
            mask = np.sum((cur_point_set>=(curmin-0.0))*(cur_point_set<(curmax+0.0)),axis=1)==3
            vidx = np.ceil((cur_point_set[mask,:2]-curmin[:2])/(curmax[:2]-curmin[:2])*[31.0,31.0])
            vidx = np.unique(vidx[:,0]*31.0+vidx[:,1])
            isvalid = np.sum(cur_semantic_seg>-1)/1.0/len(cur_semantic_seg)>=0.7 and len(vidx)/31.0/31.0>=0.3
#             print('isvalid', isvalid,len(vidx)/31.0/31.0,np.sum(cur_semantic_seg>-1),len(cur_semantic_seg))
            if isvalid:
                break
        choice = np.random.choice(len(cur_semantic_seg), npoints, replace=True)
        point_set = cur_point_set[choice,:]
        feature_set = cur_feat_set[choice,:]
        semantic_seg = cur_semantic_seg[choice]
        mask = mask[choice]
        sample_weight = labelweights[semantic_seg]
        sample_weight *= mask
        return point_set, semantic_seg, sample_weight,feature_set
    
    if(dataset =='test'):

        cur_point_set = test_xyz[index]
        cur_semantic_seg = test_label[index].astype(np.int32)
        feature_set = test_feats[index]

        point_set = pc_normalize_min(cur_point_set)
        semantic_seg = cur_semantic_seg # N
        sample_weight = labelweights_t[semantic_seg]
    
        point_sets = np.expand_dims(point_set,0) # 1xNx3
        feature_set = np.expand_dims(feature_set,0) # 1xNx3
        semantic_segs = np.expand_dims(semantic_seg,0)  # 1xN
        sample_weights = np.expand_dims(sample_weight,0)  # 1xN
        return point_sets, semantic_segs, sample_weights,feature_set
    
def get_batch_wdp(dataset, batch_idx):
    bsize = BATCH_SIZE
    batch_data = np.zeros((bsize, NUM_POINT, 3))
    batch_feats = np.zeros((bsize, NUM_POINT, 1))
    batch_label = np.zeros((bsize, NUM_POINT), dtype=np.int32)
    batch_smpw = np.zeros((bsize, NUM_POINT), dtype=np.float32)
    for i in range(bsize):
        ps,seg,smpw,feat = get_batch('train',index=0)
        ps = pc_normalize_min(ps)
        batch_data[i,...] = ps
        batch_label[i,:] = seg
        batch_smpw[i,:] = smpw
        batch_feats[i,:] = feat

        dropout_ratio = np.random.random()*0.875 # 0-0.875
        drop_idx = np.where(np.random.random((ps.shape[0]))<=dropout_ratio)[0]
        batch_data[i,drop_idx,:] = batch_data[i,0,:]
        batch_label[i,drop_idx] = batch_label[i,0]
        batch_smpw[i,drop_idx] *= 0
        
    return batch_data, batch_label, batch_smpw, batch_feats   

In [5]:
# train_f = open('Data/train_merge_min_norm_fea.pickle', 'rb')
# train_xyz, train_label, train_feats = pickle.load(train_f, encoding='bytes')
# train_xyz, train_label, train_feats = pickle.load(train_f)
# train_f.close()

test_f = open('Data/test_merge_min_norm_fea_paper_height.pickle', 'rb')
test_xyz, test_label, test_feats = pickle.load(test_f, encoding='bytes')
test_feats = [tt[:,1:2] for tt in test_feats] #reflectance
test_f.close()

NUM_CLASSES = 9
label_values = range(NUM_CLASSES)

trainSet = np.loadtxt('Data/train_height.pts',skiprows=1)

label_w = trainSet[:,4].astype('uint8')
trainSet[:,3] = trainSet[:,3]/trainSet[:,3].max() #height above ground
trainSet[:,5] = trainSet[:,5]/trainSet[:,5].max() #reflectance

# trainFeats = trainSet[:,[3,5]] #use reflectance and height above ground
trainFeats = trainSet[:,5:6] #only use reflectance

labelweights = np.zeros(9)
tmp,_ = np.histogram(label_w,range(10))
labelweights = tmp
labelweights = labelweights.astype(np.float32)
labelweights = labelweights/np.sum(labelweights)
labelweights = 1/np.log(1.4+labelweights)
print(labelweights)

labelweights_t = np.ones(9)
print(labelweights_t)

[2.967452  2.0217078 1.9802364 2.9339767 2.874856  2.1228907 2.7627099
 2.6275027 2.1882594]
[1. 1. 1. 1. 1. 1. 1. 1. 1.]


In [6]:
def train_one_epoch(sess, ops, train_writer):
    """ ops: dict mapping from string to tf ops """
    is_training = True
    
    log_string('----')
    
   # Shuffle train samples
    train_idxs = np.arange(0, len(train_xyz))
    np.random.shuffle(train_idxs)
    num_batches = len(train_xyz)//BATCH_SIZE
    
    total_correct = 0
    total_seen = 0
    loss_sum = 0
    
    for batch_idx in range(num_batches):
        
        start_idx = batch_idx * BATCH_SIZE
        end_idx = (batch_idx+1) * BATCH_SIZE
        
        batch_data, batch_label, batch_smpw, batch_feats = get_batch_wdp('train', batch_idx)
        
        if batch_idx % (num_batches/2) == 0:
            print('Current batch/total batch num: %d/%d'%(batch_idx,num_batches))
        
        aug_data = provider.rotate_point_cloud_z(batch_data)
        
        feed_dict = {ops['pointclouds_pl']: aug_data,
                     ops['feature_pl']: batch_feats,
                     ops['labels_pl']: batch_label,
                     ops['smpws_pl']: batch_smpw,
                     ops['is_training_pl']: is_training,}
        summary, step, _, loss_val, pred_val, lr_val = sess.run([ops['merged'], ops['step'], ops['train_op'], ops['loss'], ops['pred'], ops['learnrate']],
                                         feed_dict=feed_dict)
        train_writer.add_summary(summary, step)
        pred_val = np.argmax(pred_val, 2)
        correct = np.sum(pred_val == batch_label)
        total_correct += correct
        total_seen += (BATCH_SIZE*NUM_POINT)
        loss_sum += loss_val
        
    log_string('learn rate: %f' % (lr_val))
    log_string('mean loss: %f' % (loss_sum / float(num_batches)))
    log_string('accuracy: %f' % (total_correct / float(total_seen)))
    
    mloss = loss_sum / float(num_batches)
    macc = total_correct / float(total_seen)
    return mloss, macc

def eval_one_epoch_whole_scene(sess, ops):
    """ ops: dict mapping from string to tf ops """
    is_training = False
    total_correct = 0
    total_seen = 0
    loss_sum = 0
    total_seen_class = [0 for _ in range(NUM_CLASSES)]
    total_correct_class = [0 for _ in range(NUM_CLASSES)]
    
    log_string('----')
    
    test_idxs = np.arange(0, len(test_xyz))
    
    TEST_BATCH_SIZE = 1
    num_batches = len(test_xyz)
    
    Confs = []
    
    
    is_continue_batch = False
    
    for batch_idx in range(num_batches):
        
        batch_data, batch_label, batch_smpw, batch_feats = get_batch('test', batch_idx)
        
#         print('Current start end /total batch num: %d %d/%d'%(start_idx, end_idx, num_batches))
        
        aug_data = batch_data
        
#         aug_data = provider.rotate_point_cloud_z(batch_data)
        
        feed_dict = {ops['pointclouds_pl']: aug_data,
                     ops['feature_pl']: batch_feats,
                     ops['labels_pl']: batch_label,
                     ops['smpws_pl']: batch_smpw,
                     ops['is_training_pl']: is_training}
        print("feed_dict: ",feed_dict)
        summary, step, loss_val, pred_val, lr_val = sess.run([ops['merged'], ops['step'], ops['loss'], ops['pred'], ops['learnrate']],
                                      feed_dict=feed_dict)
        
        pred_val = np.argmax(pred_val, 2)
        correct = np.sum(pred_val == batch_label)
        total_correct += correct
        total_seen += batch_data.shape[1]
        loss_sum += loss_val
        
        NUM_POINT_fact = batch_data.shape[1]
        for i in range(TEST_BATCH_SIZE):
            for j in range(NUM_POINT_fact):
                l = batch_label[i, j]
                total_seen_class[l] += 1
                total_correct_class[l] += (pred_val[i, j] == l)
                
        from sklearn.metrics import confusion_matrix
        Confs += [confusion_matrix(batch_label.flatten(), pred_val.flatten(), label_values)]
        
    C = np.sum(np.stack(Confs), axis=0).astype(np.float32)
    oa, avgF1 = Acc_from_confusions(C)
    
    log_string('learn rate: %f' % (lr_val))
    log_string('eval mean loss: %f' % (loss_sum / float(num_batches)))
    log_string('eval accuracy: %f'% (total_correct / float(total_seen)))
    log_string('eval avg class acc: %f' % (np.mean(np.array(total_correct_class)/np.array(total_seen_class,dtype=np.float))))
    
    mloss = loss_sum / float(num_batches)
    macc = total_correct / float(total_seen)
    return oa, avgF1

In [7]:
# Just clears out previous sessions
tf.reset_default_graph()
K.clear_session()

# Run the session, add the parameters to the graph, testing is run in the eval_one_epoch_whole_scene line
with tf.Session() as sess:
    pointclouds_pl, labels_pl, smpws_pl = placeholder_inputs(BATCH_SIZE, NUM_POINT)
    feature_pl = tf.placeholder(tf.float32, shape=(None, None, 1))
    is_training_pl = tf.placeholder(tf.bool, shape=())

    # Note the global_step=batch parameter to minimize. 
    # That tells the optimizer to helpfully increment the 'batch' parameter for you every time it trains.
    batch = tf.Variable(0)
    bn_decay = get_bn_decay(batch)
    tf.summary.scalar('bn_decay', bn_decay)

    # Get model and loss 
    pred, end_points = MODEL.get_model(pointclouds_pl, is_training_pl, NUM_CLASSES, bn_decay=bn_decay, feature=feature_pl)
    loss = MODEL.get_loss(pred, labels_pl, smpws_pl)

    tf.summary.scalar('loss', loss)

    correct = tf.equal(tf.argmax(pred, 2), tf.to_int64(labels_pl))
    accuracy = tf.reduce_sum(tf.cast(correct, tf.float32)) / float(BATCH_SIZE*NUM_POINT)
    tf.summary.scalar('accuracy', accuracy)

    # Get training operator
    learning_rate = get_learning_rate(batch)
    tf.summary.scalar('learning_rate', learning_rate)
    if OPTIMIZER == 'momentum':
        optimizer = tf.train.MomentumOptimizer(learning_rate, momentum=MOMENTUM)
    elif OPTIMIZER == 'adam':
        optimizer = tf.train.AdamOptimizer(learning_rate)
    train_op = optimizer.minimize(loss, global_step=batch)
    
    # Add summary writers
    merged = tf.summary.merge_all()
    
    # Add ops to save and restore all the variables.
    saver = tf.train.Saver()
    ops = {'pointclouds_pl': pointclouds_pl,
       'labels_pl': labels_pl,
       'feature_pl': feature_pl,
       'smpws_pl': smpws_pl,
       'is_training_pl': is_training_pl,
       'pred': pred,
       'loss': loss,
       'train_op': train_op,
       'merged': merged,
       'step': batch,
       'learnrate': learning_rate}
    
    saver = tf.train.import_meta_graph(os.path.join(LOG_DIR, 'best_model_cleared.ckpt.meta'))
    saver.restore(sess, tf.train.latest_checkpoint(LOG_DIR))

    # This runs the testing
    eval_one_epoch_whole_scene(sess, ops)

# Not sure we need this, but clears the session
K.clear_session()


INFO:tensorflow:Restoring parameters from log_wen_v16_sample8192_group2_lw14_F1_noheight/best_model_cleared.ckpt
----
feed_dict:  {<tf.Tensor 'Placeholder:0' shape=(?, ?, 3) dtype=float32>: array([[[ 1.59375   ,  9.5       ,  0.13000488],
        [ 1.59375   ,  9.5       ,  0.10998535],
        [ 1.59375   ,  9.5       ,  0.13998413],
        ...,
        [29.9375    ,  4.        , 12.32000732],
        [29.9375    ,  4.        , 12.38000488],
        [29.90625   ,  3.5       , 12.51000977]]]), <tf.Tensor 'Placeholder_3:0' shape=(?, ?, 1) dtype=float32>: array([[[0.047059],
        [0.058824],
        [0.05098 ],
        ...,
        [0.003922],
        [0.070588],
        [0.058824]]]), <tf.Tensor 'Placeholder_1:0' shape=(?, ?) dtype=int32>: array([[2, 2, 2, ..., 5, 5, 5]], dtype=int32), <tf.Tensor 'Placeholder_2:0' shape=(?, ?) dtype=float32>: array([[1., 1., 1., ..., 1., 1., 1.]]), <tf.Tensor 'Placeholder_4:0' shape=() dtype=bool>: False}


/home/michael/python-virtual-environments/pointscnn/lib/python3.6/site-packages/sklearn/utils/validation.py:71: FutureWarning: Pass labels=range(0, 9) as keyword args. From version 0.25 passing these as positional arguments will result in an error
  FutureWarning)


feed_dict:  {<tf.Tensor 'Placeholder:0' shape=(?, ?, 3) dtype=float32>: array([[[ 0.03125   ,  2.5       ,  0.41000366],
        [ 0.03125   ,  2.5       ,  0.51000976],
        [ 0.0625    ,  2.5       ,  0.45999145],
        ...,
        [29.90625   ,  5.        , 10.16998291],
        [29.875     ,  4.5       , 10.58999634],
        [29.84375   ,  4.        , 11.04000854]]]), <tf.Tensor 'Placeholder_3:0' shape=(?, ?, 1) dtype=float32>: array([[[0.007843],
        [0.011765],
        [0.043137],
        ...,
        [0.545098],
        [0.509804],
        [0.486275]]]), <tf.Tensor 'Placeholder_1:0' shape=(?, ?) dtype=int32>: array([[2, 2, 2, ..., 5, 5, 5]], dtype=int32), <tf.Tensor 'Placeholder_2:0' shape=(?, ?) dtype=float32>: array([[1., 1., 1., ..., 1., 1., 1.]]), <tf.Tensor 'Placeholder_4:0' shape=() dtype=bool>: False}
feed_dict:  {<tf.Tensor 'Placeholder:0' shape=(?, ?, 3) dtype=float32>: array([[[ 0.375     , 22.        ,  4.0499878 ],
        [ 0.375     , 21.5       ,  4.079

feed_dict:  {<tf.Tensor 'Placeholder:0' shape=(?, ?, 3) dtype=float32>: array([[[ 0.15625   , 29.5       ,  2.05001831],
        [ 0.125     , 29.        ,  2.14001465],
        [ 0.1875    , 28.5       ,  0.97000122],
        ...,
        [40.28125   ,  2.        ,  0.18002319],
        [40.875     ,  0.5       ,  0.26000977],
        [40.875     ,  0.        ,  0.29000855]]]), <tf.Tensor 'Placeholder_3:0' shape=(?, ?, 1) dtype=float32>: array([[[0.686275],
        [0.192157],
        [0.294118],
        ...,
        [0.239216],
        [0.156863],
        [0.152941]]]), <tf.Tensor 'Placeholder_1:0' shape=(?, ?) dtype=int32>: array([[8, 8, 2, ..., 2, 2, 2]], dtype=int32), <tf.Tensor 'Placeholder_2:0' shape=(?, ?) dtype=float32>: array([[1., 1., 1., ..., 1., 1., 1.]]), <tf.Tensor 'Placeholder_4:0' shape=() dtype=bool>: False}
feed_dict:  {<tf.Tensor 'Placeholder:0' shape=(?, ?, 3) dtype=float32>: array([[[3.12500000e-02, 2.95000000e+01, 9.87997437e+00],
        [3.12500000e-02, 2.90000

feed_dict:  {<tf.Tensor 'Placeholder:0' shape=(?, ?, 3) dtype=float32>: array([[[0.        , 8.5       , 0.25      ],
        [0.        , 9.        , 0.25997925],
        [0.        , 9.5       , 0.26998902],
        ...,
        [0.        , 2.        , 0.16998292],
        [0.        , 2.        , 0.17999268],
        [0.        , 2.        , 0.16998292]]]), <tf.Tensor 'Placeholder_3:0' shape=(?, ?, 1) dtype=float32>: array([[[0.117647],
        [0.113725],
        [0.117647],
        ...,
        [0.086275],
        [0.078431],
        [0.066667]]]), <tf.Tensor 'Placeholder_1:0' shape=(?, ?) dtype=int32>: array([[2, 2, 2, ..., 2, 2, 2]], dtype=int32), <tf.Tensor 'Placeholder_2:0' shape=(?, ?) dtype=float32>: array([[1., 1., 1., ..., 1., 1., 1.]]), <tf.Tensor 'Placeholder_4:0' shape=() dtype=bool>: False}
feed_dict:  {<tf.Tensor 'Placeholder:0' shape=(?, ?, 3) dtype=float32>: array([[[ 0.4375    , 29.5       ,  6.8500061 ],
        [ 0.4375    , 29.        ,  6.73999023],
        [ 

feed_dict:  {<tf.Tensor 'Placeholder:0' shape=(?, ?, 3) dtype=float32>: array([[[ 0.5625    ,  3.        ,  0.88998414],
        [ 0.5625    ,  3.        ,  0.77999878],
        [ 0.5625    ,  3.5       ,  0.75      ],
        ...,
        [26.84375   ,  6.5       ,  0.08999634],
        [27.3125    ,  5.        ,  0.1499939 ],
        [27.34375   ,  4.5       ,  1.33999634]]]), <tf.Tensor 'Placeholder_3:0' shape=(?, ?, 1) dtype=float32>: array([[[0.160784],
        [0.011765],
        [0.101961],
        ...,
        [0.156863],
        [0.12549 ],
        [0.490196]]]), <tf.Tensor 'Placeholder_1:0' shape=(?, ?) dtype=int32>: array([[1, 2, 2, ..., 2, 2, 4]], dtype=int32), <tf.Tensor 'Placeholder_2:0' shape=(?, ?) dtype=float32>: array([[1., 1., 1., ..., 1., 1., 1.]]), <tf.Tensor 'Placeholder_4:0' shape=() dtype=bool>: False}
feed_dict:  {<tf.Tensor 'Placeholder:0' shape=(?, ?, 3) dtype=float32>: array([[[ 0.        , 10.5       ,  5.19000244],
        [ 0.15625   , 11.        ,  2.059

feed_dict:  {<tf.Tensor 'Placeholder:0' shape=(?, ?, 3) dtype=float32>: array([[[ 0.        ,  0.5       ,  7.0699768 ],
        [ 0.03125   ,  0.        ,  7.54998779],
        [ 0.625     ,  0.        ,  7.35998535],
        ...,
        [29.84375   , 28.5       ,  3.26998901],
        [29.84375   , 29.        ,  3.26998901],
        [29.84375   , 29.5       ,  3.25      ]]]), <tf.Tensor 'Placeholder_3:0' shape=(?, ?, 1) dtype=float32>: array([[[0.015686],
        [0.329412],
        [0.678431],
        ...,
        [0.258824],
        [0.007843],
        [0.003922]]]), <tf.Tensor 'Placeholder_1:0' shape=(?, ?) dtype=int32>: array([[8, 8, 8, ..., 1, 2, 2]], dtype=int32), <tf.Tensor 'Placeholder_2:0' shape=(?, ?) dtype=float32>: array([[1., 1., 1., ..., 1., 1., 1.]]), <tf.Tensor 'Placeholder_4:0' shape=() dtype=bool>: False}
feed_dict:  {<tf.Tensor 'Placeholder:0' shape=(?, ?, 3) dtype=float32>: array([[[30.625     ,  6.5       ,  0.36999511],
        [30.59375   ,  7.        ,  0.429

feed_dict:  {<tf.Tensor 'Placeholder:0' shape=(?, ?, 3) dtype=float32>: array([[[ 0.15625   ,  0.        ,  3.5499878 ],
        [ 0.15625   ,  0.        ,  3.66998291],
        [ 0.125     ,  0.5       ,  3.63998413],
        ...,
        [29.9375    ,  8.5       ,  6.00997925],
        [29.96875   ,  7.5       ,  3.63000489],
        [29.9375    ,  7.5       ,  5.5       ]]]), <tf.Tensor 'Placeholder_3:0' shape=(?, ?, 1) dtype=float32>: array([[[0.380392],
        [0.419608],
        [0.388235],
        ...,
        [0.172549],
        [0.384314],
        [0.141176]]]), <tf.Tensor 'Placeholder_1:0' shape=(?, ?) dtype=int32>: array([[7, 7, 7, ..., 8, 8, 8]], dtype=int32), <tf.Tensor 'Placeholder_2:0' shape=(?, ?) dtype=float32>: array([[1., 1., 1., ..., 1., 1., 1.]]), <tf.Tensor 'Placeholder_4:0' shape=() dtype=bool>: False}
feed_dict:  {<tf.Tensor 'Placeholder:0' shape=(?, ?, 3) dtype=float32>: array([[[ 0.125     ,  2.5       ,  0.1000061 ],
        [ 0.125     ,  3.        ,  0.100